<a href="https://colab.research.google.com/github/Taaniya/exploring-langGraph/blob/main/Stream_custom_data_with_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.1 MB/s eta 0:00:00


In [9]:
from langchain_core.messages import AIMessage, HumanMessage

from langgraph.graph import START, StateGraph, MessagesState, END
from langgraph.types import StreamWriter

In [5]:
MessagesState

langgraph.graph.message.MessagesState

In [6]:
async def my_node(
    state: MessagesState,
    writer: StreamWriter         # <--- provide StreamWriter to write chunks to be streamed
):

  chunks = [
      "Four",
      "score",
      "and",
      "seven",
      "years",
      "ago",
      "our",
      "fathers",
      "..."
  ]
  for chunk in chunks:
    # write the chunk to be streamed using stream_mode = custom
    writer(chunk)

  return {"messages": [AIMessage(content=" ".join(chunks))]}

In [8]:
# Define the new graph
workflow = StateGraph(MessagesState)

workflow.add_node("model", my_node)
workflow.add_edge(START, "model")
workflow.add_edge("model", END)

app = workflow.compile()

stream_mode = custom, only streams custom messages from within the graph

In [12]:
inputs = [HumanMessage(content="what are you thinking about?")]

async for chunk in app.astream({"messages": inputs}, stream_mode='custom'):   # custom - only streams custom messages
  print(type(chunk))
  print(chunk, flush=True)

<class 'str'>
Four
<class 'str'>
score
<class 'str'>
and
<class 'str'>
seven
<class 'str'>
years
<class 'str'>
ago
<class 'str'>
our
<class 'str'>
fathers
<class 'str'>
...


stream_mode = [custom, values], streams both custom messages & state value of graph at every step.

In [13]:
inputs = [HumanMessage(content="what are you thinking about?")]

async for chunk in app.astream({"messages": inputs}, stream_mode=['custom', 'values']):   # custom - only streams custom messages
  print(type(chunk))
  print(chunk, flush=True)

<class 'tuple'>
('values', {'messages': [HumanMessage(content='what are you thinking about?', additional_kwargs={}, response_metadata={}, id='4aa72041-7a10-4e25-93af-0870b6f21907')]})
<class 'tuple'>
('custom', 'Four')
<class 'tuple'>
('custom', 'score')
<class 'tuple'>
('custom', 'and')
<class 'tuple'>
('custom', 'seven')
<class 'tuple'>
('custom', 'years')
<class 'tuple'>
('custom', 'ago')
<class 'tuple'>
('custom', 'our')
<class 'tuple'>
('custom', 'fathers')
<class 'tuple'>
('custom', '...')
<class 'tuple'>
('values', {'messages': [HumanMessage(content='what are you thinking about?', additional_kwargs={}, response_metadata={}, id='4aa72041-7a10-4e25-93af-0870b6f21907'), AIMessage(content='Four score and seven years ago our fathers ...', additional_kwargs={}, response_metadata={}, id='5d0af646-935f-4398-bfff-1fd9113824b1')]})


#### References

* https://langchain-ai.github.io/langgraph/how-tos/streaming/
* https://langchain-ai.github.io/langgraph/how-tos/streaming-subgraphs/
* https://python.langchain.com/docs/concepts/streaming/#overview
* https://langchain-ai.github.io/langgraph/reference/types/#langgraph.types.StreamWriter


